<a href="https://colab.research.google.com/github/NissuGeorge/testing/blob/master/DGM_Assignment_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from keras.datasets import fashion_mnist



In [19]:
def preprocess_data(X):
    # Subtract the mean and divide by the standard deviation of each feature
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X

In [20]:
def initialize_parameters(X, K):
    # Initialize the means randomly
    means = np.random.randn(K, X.shape[1])
    
    # Initialize the covariances to the identity matrix
    covariances = np.array([np.eye(X.shape[1]) for i in range(K)])
    
    # Initialize the mixing coefficients uniformly
    mixing_coeffs = np.ones(K) / K
    
    return means, covariances, mixing_coeffs

In [21]:
def e_step(X, means, covariances, mixing_coeffs):
    # Compute the posterior probabilities of each sample belonging to each component
    posteriors = np.zeros((X.shape[0], means.shape[0]))
    for k in range(means.shape[0]):
        posteriors[:, k] = mixing_coeffs[k] * multivariate_normal.pdf(X, means[k], covariances[k])
    posteriors /= np.sum(posteriors, axis=1, keepdims=True)+ 1e-6
    
    return posteriors

In [22]:
def m_step(X, posteriors):
    # Update the means, covariances, and mixing coefficients of each component
    means = np.dot(posteriors.T, X) / (np.sum(posteriors, axis=0, keepdims=True).T + 1e-6)
    covariances = np.zeros((posteriors.shape[1], X.shape[1], X.shape[1]))
    mixing_coeffs = np.mean(posteriors, axis=0)
    for k in range(posteriors.shape[1]):
        diff = X - means[k]
        covariances[k] = np.dot(posteriors[:, k] * diff.T, diff) / (np.sum(posteriors[:, k])+ 1e-6)
        covariances[k] += np.eye(covariances[k].shape[0]) * 1e-6
    
    return means, covariances, mixing_coeffs

In [23]:
def fit_gmm(X, K, n_iterations=100):
    X = preprocess_data(X)
    means, covariances, mixing_coeffs = initialize_parameters(X, K)
    print(means)
    print(covariances)
    print(mixing_coeffs)
    best_likelihood = -np.inf
    log_likelihoods=[]
    for i in range(n_iterations):
        posteriors = e_step(X, means, covariances, mixing_coeffs)
        means, covariances, mixing_coeffs = m_step(X, posteriors)
        log_likelihood = log_likelihood(X, means, covariances, mixing_coeffs)
        log_likelihoods.append(log_likelihood)
    
    # Print progress
        print(f'Iteration {i+1}/{n_iterations}: log-likelihood={log_likelihood}')

        likelihood = np.sum(np.log(np.dot(posteriors, mixing_coeffs)+1e-6))
        if likelihood > best_likelihood:
            best_means = means
            best_covariances = covariances
            best_mixing_coeffs = mixing_coeffs
            best_likelihood = likelihood
    
    return best_means, best_covariances, best_mixing_coeffs

In [24]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], -1)) / 255.0
#x_train=preprocess_data(x_train)
x_test = x_test.reshape((x_test.shape[0], -1)) / 255.0

#x_train[np.isnan(x_train)] = 0
#x_train[np.isinf(x_train)] = 0



4422102/4422102 [==============================] - 1s 0us/step


In [10]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
print(y_train)

[9 0 0 ... 3 0 5]


In [27]:
# Fit a 10-component GMM on the Fashion MNIST dataset
K = 10
means, covariances, mixing_coeffs = fit_gmm(x_train, K)

print(means)

[[-0.09001597 -0.03889421  1.83934891 ...  0.67830189  0.1551299
  -0.50540595]
 [-1.17732143  0.32178667 -0.80701612 ...  0.03304286  0.76947036
  -0.54371207]
 [ 0.91231957 -0.92388972 -0.30967512 ...  1.39823131 -0.75625811
  -2.56762964]
 ...
 [ 1.79119698 -0.03298685 -0.19760018 ... -0.38908376 -0.35115741
   1.31896576]
 [-0.45762727  0.29413511 -1.69520259 ...  2.45569166 -0.72558007
   0.37417757]
 [ 1.60415234 -0.51746332 -0.73023531 ...  0.3891452   1.49932302
   1.27837895]]
[[[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[1. 0. 0.

UnboundLocalError: ignored

In [9]:
print(means)
print(covariances)
print(mixing_coeffs)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[[1.e-06 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 1.e-06 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 1.e-06 ... 0.e+00 0.e+00 0.e+00]
  ...
  [0.e+00 0.e+00 0.e+00 ... 1.e-06 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 ... 0.e+00 1.e-06 0.e+00]
  [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 1.e-06]]

 [[1.e-06 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 1.e-06 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 1.e-06 ... 0.e+00 0.e+00 0.e+00]
  ...
  [0.e+00 0.e+00 0.e+00 ... 1.e-06 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 ... 0.e+00 1.e-06 0.e+00]
  [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 1.e-06]]

 [[1.e-06 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 1.e-06 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 1.e-06 ... 0.e+00 0.e+00 0.e+00]
  ...
  [0.e+00 0.e+00 0.e+00 ... 1.e-06 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+0

In [29]:

posteriors = e_step(x_test, means, covariances, mixing_coeffs)
predicted_labels = np.argmax(posteriors, axis=1)
accuracy = np.mean(predicted_labels == y_test)
print(f'accuracy:{accuracy}')

accuracy:0.1


In [6]:
import numpy as np
from scipy.stats import multivariate_normal

def log_likelihood(X, means, covariances, mixing_coeffs):
    """Compute the log-likelihood of the data given the GMM parameters.

    Parameters:
    -----------
    X : numpy array
        The data points, with shape (num_samples, num_features).
    means : numpy array
        The mean vectors for each component, with shape (num_components, num_features).
    covariances : numpy array
        The covariance matrices for each component, with shape (num_components, num_features, num_features).
    mixing_coeffs : numpy array
        The mixing coefficients for each component, with shape (num_components,).

    Returns:
    --------
    log_likelihood : float
        The log-likelihood of the data given the GMM parameters.
    """
 
    num_components = means.shape[0]
    num_samples = X.shape[0]
    likelihoods = np.zeros((num_samples, num_components))
    for k in range(num_components):
        likelihoods[:, k] = multivariate_normal.pdf(X, mean=means[k], cov=covariances[k])
    log_likelihood = np.sum(np.log(np.dot(likelihoods, mixing_coeffs)+ 1e-6))
    return log_likelihood


In [33]:
log_likelihood_score=log_likelihood(x_test, means, covariances, mixing_coeffs)

In [34]:
print(log_likelihood_score)

-138155.10557964272


In [43]:
# Generate 16 new images from the GMM

# Add a small constant to the mixing coefficients to avoid NaNs
smoothing_const = 1e-8
mixing_coeffs += smoothing_const
print("Mixing coefficients:")
print(mixing_coeffs)

# Normalize the mixing coefficients to sum to 1
mixing_coeffs = mixing_coeffs/np.sum(mixing_coeffs)
print("Mixing coefficients:")
print(mixing_coeffs)



Mixing coefficients:
[nan nan nan nan nan nan nan nan nan nan]
Mixing coefficients:
[nan nan nan nan nan nan nan nan nan nan]


In [ ]:
num_samples = 16
num_pixels=x_train.shape[1]
num_components = means.shape[0]
new_X = np.zeros((num_samples, num_pixels))
for i in range(num_samples):
    # Sample a component index from the mixing coefficients
    k = np.random.choice(num_components, p=mixing_coeffs)
    
    # Sample a data point from the selected component
    new_X[i, :] = np.random.multivariate_normal(mean=means[k], cov=covariances[k])

# Reshape the data points into image format
new_images = new_X.reshape((-1, 28, 28))

# Display the new images
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8))
for i in range(num_samples):
    ax = axes.flat[i]
    ax.imshow(new_images[i], cmap='gray')
    ax.axis('off')
plt.show()


In [12]:
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.datasets import fetch_openml
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from keras.datasets import fashion_mnist

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], -1)) / 255.0
#x_train=preprocess_data(x_train)
x_test = x_test.reshape((x_test.shape[0], -1)) / 255.0
X=(x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)

def initialize_parameters(X, n_clusters):

  n_samples, n_features = X.shape

  # Initialize means randomly
  kmeans = KMeans(n_clusters)
  kmeans.fit(X)
  means = kmeans.cluster_centers_
  #means = np.random.randn(n_clusters, n_features)

  # Initialize covariances randomly
  #covariances = np.zeros((n_clusters, n_features, n_features))
  #for i in range(n_clusters):
    #covariances[i] = np.eye(n_features)*1e-6
  covariances = np.tile(np.diag(np.ones(n_features)), (n_clusters, 1, 1))
  for j in range(n_clusters):
    covariances[j] += 1e-2* np.eye(n_features)
    # Initialize weights uniformly
  weights = np.ones(n_clusters) / n_clusters

  return means, covariances, weights

def log_Likelihood(X, means, covariances, mixing_coeffs):
  num_components = means.shape[0]
  num_samples = X.shape[0]
  likelihoods = np.zeros((num_samples, num_components))
  for k in range(num_components):
    likelihoods[:, k] = multivariate_normal.pdf(X, mean=means[k], cov=covariances[k])
  loglikelihood = np.sum(np.log(np.dot(likelihoods, mixing_coeffs)+ 1e-6))
  return loglikelihood


def fit_gmm(X, n_clusters, n_iterations=100, tolerance=1e-4):
  n_samples, n_features = X.shape

  # Initialize parameters
  means, covariances, weights = initialize_parameters(X, n_clusters)
  print('Initial Means:',means)
  print('Initial Covariances:',covariances)
  print('Initial Weights:',weights)

  # Initialize log-likelihood
  prev_log_likelihood = -np.inf

  # Loop over iterations
  for i in range(n_iterations):
    print('ITERATION:',i)
    # E-step: calculate the responsibilities
    responsibilities = np.zeros((n_samples, n_clusters))
    for j in range(n_clusters):
      #print('Cluster:',j)
      #print("multivariate normal:",multivariate_normal.pdf(X, means[j], covariances[j]))
      #print('weight:',weights[j])
      responsibilities[:, j] = weights[j] * multivariate_normal.pdf(X, means[j], covariances[j])+1e-6
      #print('Responsibilities:',responsibilities)
    responsibilities /= (responsibilities.sum(axis=1, keepdims=True)+1e-6)
    print('Normalized Responsibilities:',responsibilities)

    # M-step: update the parameters
    for j in range(n_clusters):
      # Update the mean
      means[j] = np.average(X, axis=0, weights=responsibilities[:, j])
      #print('means:',means[j])
      # Update the covariance
      covariances[j] = np.cov(X.T, aweights=responsibilities[:, j], ddof=0)
      #print('covariances:',covariances[j])

      # Update the weight
      weights[j] = responsibilities[:, j].mean()
      #print('weights:',weights[j])
    #print('Means:',means)
    #print('covariances:',covariances)
    #print('weights:',weights)
    # Calculate the log-likelihood
    log_likelihood = log_Likelihood(X,means,covariances,weights)
    #np.sum(np.log(np.sum(np.fromiter((weights[j] * multivariate_normal.pdf(X, means[j], covariances[j]) for j in range(n_clusters)),dtype=float64))))
    print('log_likelihood:', log_likelihood)

    # Check for convergence
    if np.abs(log_likelihood - prev_log_likelihood) < tolerance:
      break
    prev_log_likelihood = log_likelihood

  return means, covariances, weights, log_likelihood

# Fit the GMM to the Fashion-MNIST dataset
n_clusters = 10
means, covariances, weights, log_likelihood = fit_gmm(X, n_clusters)

# Print the log-likelihood
print('Means:',means)
print('covariances:',covariances)
print('weights:',weights)
print('Log-likelihood:', log_likelihood)


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initial Means: [[-8.64371176e-03  3.00591435e-02  4.35586546e-02 ...  4.77595566e-01
   4.36957634e-01  3.28852140e-01]
 [ 2.92060096e-02  2.33056712e-02  6.80324173e-02 ... -1.53993008e-01
  -8.85224039e-02 -3.25895259e-02]
 [-7.47904218e-03 -7.20783106e-03  5.49862046e-04 ...  1.95470883e-01
  -9.49932125e-03 -2.95775972e-02]
 ...
 [-5.94153852e-03  1.04196107e-02  2.40080472e-02 ... -4.45348671e-02
  -6.13864349e-02 -3.05328920e-02]
 [ 1.02455185e-02  6.97944687e-02  7.92144653e-02 ... -6.02744815e-02
   1.53746713e-03  4.92306761e-02]
 [-4.02239452e-03  3.68898575e-02 -1.57831296e-02 ...  7.64566046e-01
   7.09779348e-01  2.13521967e-01]]
Initial Covariances: [[[1.01 0.   0.   ... 0.   0.   0.  ]
  [0.   1.01 0.   ... 0.   0.   0.  ]
  [0.   0.   1.01 ... 0.   0.   0.  ]
  ...
  [0.   0.   0.   ... 1.01 0.   0.  ]
  [0.   0.   0.   ... 0.   1.01 0.  ]
  [0.   0.   0.   ... 0.   0.   1.01]]

 [[1.01 0.   0.   ... 0.   0.   0.  ]
  [0.   1.01 0.   ... 0.   0.   0.  ]
  [0.   0.   1.0